# Import dependencies

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split

In [ ]:
#introduction to machine learning would not be fun without deep learning
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K

# Load dataset

In [ ]:
train_data = np.array(pd.read_csv('train.csv'))[:,0]
train_solution = np.array(pd.read_csv('train.csv'))[:,1]
test_data = np.array(pd.read_csv('test.csv'))

# Analyse dataset

In [ ]:
print(train_data.shape)
print(train_data)
pd.read_csv('train.csv').head(8)
full_data = np.array(pd.read_csv('train.csv'))
unique, count = np.unique(full_data[:,1], return_counts=True)
print(count)
print(107787/count)

(112000,)
['DKWL' 'FCHN' 'KDQP' ... 'SGHC' 'KIGT' 'PGPT']
[107787   4213]
[ 1.         25.58438168]


# Preprocess dataset

### Split & One hot encoding

In [ ]:
def split(listwords):
    length = listwords.shape[0]
    output = []
    for i in range(0,length):
        output.append(np.array([char for char in listwords[i]]))
    
    return np.array(output)

def onehotencode(data):
    encoder = OneHotEncoder(sparse=False)
    return encoder.fit_transform(data)

encoded_train_X = onehotencode(split(train_data))

### Split for training

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(encoded_train_X,train_solution,test_size = 0.0001, random_state = 42)
X_train = encoded_train_X
y_train = train_solution
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(112000, 80)
(112000,)
(12, 80)
(12,)


# Train dataset

In [ ]:
## source code
#https://stackoverflow.com/questions/59963911/how-to-write-a-custom-f1-loss-function-with-weighted-average-for-keras
## This code is just a copy paste but we are allowed to use publicy available code so it's fine

def f1_weighted(true, pred):
    ground_positives = K.sum(true, axis=0) + K.epsilon()       # = TP + FN
    pred_positives = K.sum(pred, axis=0) + K.epsilon()         # = TP + FP
    true_positives = K.sum(true * pred, axis=0) + K.epsilon()  # = TP
    
    precision = true_positives / pred_positives 
    recall = true_positives / ground_positives

    f1 = 2 * (precision * recall) / (precision + recall + K.epsilon())

    weighted_f1 = f1 * ground_positives / K.sum(ground_positives) 
    weighted_f1 = K.sum(weighted_f1)

    return weighted_f1

In [ ]:
# Model declaration
model = Sequential()
model.add(Dense(150,input_shape=X_train[0].shape))
model.add(Activation('relu'))
model.add(Dropout(0.10))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.109))
model.add(Dense(50))
model.add(Activation('relu'))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# Compile model
model.compile(loss='binary_crossentropy',
                      optimizer='adam',
                      metrics=[f1_weighted])

In [ ]:
batch_size = 64
epochs = 300

weights = {0: 1, 1: 5}

# Fit model
model.fit(X_train.astype('int32'), y_train.astype('int32'),
                  batch_size=batch_size,
                  epochs=epochs,
                  verbose=True,
                  class_weight=weights)

# Predict
y_pred = model.predict_classes(X_test.astype('int32'))
print(f1_score(y_pred,y_test.astype('int')))

Epoch 1/300
1750/1750 [==============================] - 1s 696us/step - loss: 0.1440 - f1_weighted: 0.4923
Epoch 2/300
1750/1750 [==============================] - 1s 695us/step - loss: 0.0749 - f1_weighted: 0.6594
Epoch 3/300
1750/1750 [==============================] - 1s 713us/step - loss: 0.0583 - f1_weighted: 0.6913
Epoch 4/300
1750/1750 [==============================] - 1s 716us/step - loss: 0.0496 - f1_weighted: 0.7332
Epoch 5/300
1750/1750 [==============================] - 1s 721us/step - loss: 0.0441 - f1_weighted: 0.7376
Epoch 6/300
1750/1750 [==============================] - 1s 708us/step - loss: 0.0400 - f1_weighted: 0.7619
Epoch 7/300
1750/1750 [==============================] - 1s 710us/step - loss: 0.0354 - f1_weighted: 0.7719
Epoch 8/300
1750/1750 [==============================] - 1s 759us/step - loss: 0.0330 - f1_weighted: 0.7840
Epoch 9/300
1750/1750 [==============================] - 1s 745us/step - loss: 0.0305 - f1_weighted: 0.7864
Epoch 10/300
1750/1750 [====

1750/1750 [==============================] - 1s 753us/step - loss: 0.0069 - f1_weighted: 0.8814
Epoch 77/300
1750/1750 [==============================] - 1s 757us/step - loss: 0.0060 - f1_weighted: 0.8961
Epoch 78/300
1750/1750 [==============================] - 1s 783us/step - loss: 0.0069 - f1_weighted: 0.8827
Epoch 79/300
1750/1750 [==============================] - 1s 782us/step - loss: 0.0051 - f1_weighted: 0.8993
Epoch 80/300
1750/1750 [==============================] - 1s 777us/step - loss: 0.0056 - f1_weighted: 0.8953
Epoch 81/300
1750/1750 [==============================] - 1s 812us/step - loss: 0.0056 - f1_weighted: 0.9006
Epoch 82/300
1750/1750 [==============================] - 1s 765us/step - loss: 0.0060 - f1_weighted: 0.8987
Epoch 83/300
1750/1750 [==============================] - 1s 759us/step - loss: 0.0052 - f1_weighted: 0.9016
Epoch 84/300
1750/1750 [==============================] - 1s 754us/step - loss: 0.0053 - f1_weighted: 0.8849
Epoch 85/300
1750/1750 [========

1750/1750 [==============================] - 1s 747us/step - loss: 0.0038 - f1_weighted: 0.8959
Epoch 151/300
1750/1750 [==============================] - 1s 744us/step - loss: 0.0029 - f1_weighted: 0.9092
Epoch 152/300
1750/1750 [==============================] - 1s 718us/step - loss: 0.0041 - f1_weighted: 0.8992
Epoch 153/300
1750/1750 [==============================] - 1s 741us/step - loss: 0.0044 - f1_weighted: 0.8888
Epoch 154/300
1750/1750 [==============================] - 1s 742us/step - loss: 0.0019 - f1_weighted: 0.9123
Epoch 155/300
1750/1750 [==============================] - 1s 725us/step - loss: 0.0046 - f1_weighted: 0.8970
Epoch 156/300
1750/1750 [==============================] - 1s 717us/step - loss: 0.0042 - f1_weighted: 0.9080
Epoch 157/300
1750/1750 [==============================] - 1s 712us/step - loss: 0.0044 - f1_weighted: 0.9057
Epoch 158/300
1750/1750 [==============================] - 1s 737us/step - loss: 0.0032 - f1_weighted: 0.9104
Epoch 159/300
1750/1750 

1750/1750 [==============================] - 1s 747us/step - loss: 0.0032 - f1_weighted: 0.9017
Epoch 225/300
1750/1750 [==============================] - 1s 778us/step - loss: 0.0030 - f1_weighted: 0.8989
Epoch 226/300
1750/1750 [==============================] - 1s 769us/step - loss: 0.0023 - f1_weighted: 0.9069
Epoch 227/300
1750/1750 [==============================] - 1s 748us/step - loss: 0.0040 - f1_weighted: 0.9107
Epoch 228/300
1750/1750 [==============================] - 1s 742us/step - loss: 0.0034 - f1_weighted: 0.8955
Epoch 229/300
1750/1750 [==============================] - 1s 747us/step - loss: 0.0031 - f1_weighted: 0.9041
Epoch 230/300
1750/1750 [==============================] - 1s 743us/step - loss: 0.0024 - f1_weighted: 0.9082
Epoch 231/300
1750/1750 [==============================] - 1s 723us/step - loss: 0.0024 - f1_weighted: 0.9024
Epoch 232/300
1750/1750 [==============================] - 1s 727us/step - loss: 0.0029 - f1_weighted: 0.9212
Epoch 233/300
1750/1750 

1750/1750 [==============================] - 1s 741us/step - loss: 0.0026 - f1_weighted: 0.9042
Epoch 299/300
1750/1750 [==============================] - 1s 725us/step - loss: 0.0031 - f1_weighted: 0.9022
Epoch 300/300
1750/1750 [==============================] - 1s 741us/step - loss: 0.0028 - f1_weighted: 0.9138
Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
1.0


In [ ]:
y_pred = model.predict_classes(X_test.astype('int32'))
print(f1_score(y_pred,y_test.astype('int')))

1.0


## Output results

In [ ]:
output = model.predict_classes(onehotencode(split(test_data[:,0])))
print(output.shape)
data = pd.Series(output.reshape(-1))
data.to_csv("output.csv",index=False,header=False)

(48000, 1)
